# 06 FixMatch on CIFAR‑10 (few labels)

Now we move from toy dynamics to a modern SSL recipe.

FixMatch = *pseudo‑labels + confidence threshold + strong augmentation*.



### Why thresholding matters early

Early in training, confidence is unreliable. A high threshold keeps the state empty

until the model earns trust — that’s the stabilization mechanism.



## Step 1: Imports

We’ll run FixMatch with either CIFAR‑10 (full) or MNIST (fast dev run).



In [ ]:
from pathlib import Path
import sys
import torch
import matplotlib.pyplot as plt

sys.path.append(str(Path.cwd().parent / 'src'))

from utils.seed import set_seed
from data.mnist import get_mnist_ssl
from data.cifar10 import get_cifar10_ssl
from models.small_cnn import SmallCNN
from models.resnet18 import build_resnet18
from methods.fixmatch import run_fixmatch


## Step 2: Choose data + model

Set `FAST_DEV_RUN=False` for the real CIFAR‑10 experiment.



In [ ]:
set_seed(0)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
FAST_DEV_RUN = False  # set False to use CIFAR-10

if FAST_DEV_RUN:
    loaders = get_mnist_ssl('data', labeled_per_class=50, batch_size=128, num_workers=2, seed=0)
    model = SmallCNN()
    epochs = 2
else:
    loaders = get_cifar10_ssl('data', labeled_per_class=40, batch_size=128, num_workers=2, seed=0)
    model = build_resnet18()
    epochs = 5

optimizer = torch.optim.SGD(model.parameters(), lr=0.03, momentum=0.9, weight_decay=5e-4)
result = run_fixmatch(
    model,
    loaders.labeled,
    loaders.unlabeled,
    loaders.unlabeled_eval,
    loaders.test,
    optimizer,
    DEVICE,
    epochs=epochs,
    tau=0.95,
    lambda_u=1.0,
)


## Step 3: Train FixMatch

Watch how acceptance and accuracy evolve.



In [ ]:
plt.figure(figsize=(4.5, 3))
plt.plot([r['epoch'] for r in result.history], [r['test_acc'] for r in result.history], marker='o')
plt.title('FixMatch test accuracy')
plt.xlabel('epoch')
plt.ylabel('acc')


### Why the accuracy can dip early
In early epochs, acceptance can be *very high* while pseudo‑label accuracy is < 100%.
That means the model trains on a large volume of slightly noisy labels, which can
cause a small short‑term dip in test accuracy. This is expected in short runs.


In [ ]:
plt.figure(figsize=(4.5, 3))
plt.plot([r['epoch'] for r in result.history], [r['accept_rate'] for r in result.history], marker='o', label='accept_rate')
plt.plot([r['epoch'] for r in result.history], [r['pseudo_label_acc'] for r in result.history], marker='o', label='pseudo_label_acc')
plt.title('FixMatch acceptance vs pseudo‑label quality')
plt.xlabel('epoch')
plt.ylabel('value')
plt.legend(frameon=False)


## Observations → Why → SSL opportunity

**What you’ll likely see**

- Acceptance rate is low early, then rises as the model improves.

- Test accuracy climbs faster than a pure supervised baseline with the same labels.



**Why this happens**

- The threshold prevents noisy early pseudo‑labels.

- Strong augmentation enforces consistency.



**SSL opportunity**

- Trust can be earned gradually via confidence thresholds.

